In [1]:
# Load a random structure from PMG json.

from crystalgraph import LQG
from pymatgen.core.structure import Structure
import networkx as nx
from collections import Counter

def get_zeolite_structure_and_lqg(index: int):
    zeo = Structure.from_file("pmg_json/{:06d}.json".format(index))
    g = LQG.from_structure(zeo)
    print(">", zeo.composition)
    print(">", g.nxg)
    print(">","# of cross boundary edges", len([d for u, v, d in g.nxg.edges(data=True) if d['voltage'] != (0, 0, 0)]))
    print(">", g.properties)
    nnb_compositions = []
    for n in g.nxg.nodes:
        nbs = list(nx.neighbors(g.nxg, n))
        center_element = g.symbols[n]
        nb_elements = [g.symbols[nb] for nb in nbs]
        comp = " ".join(sorted(nb_elements + [center_element]))
        nnb_compositions.append(comp)
    print(">", Counter(nnb_compositions))
    return zeo, g

def draw_bu_contraction(lqg: LQG, output:str):
    lqg_bu = lqg.bu_contraction()
    lqg_bu.draw_graphviz(ipython=False, filename=f"{output}.png")

def bc_embedding(zeo: Structure, lqg: LQG, output: str):
    bce_structure = lqg.to_structure(zeo.lattice, barycentric=True, barycentric_dim=3)
    bce_structure.to(f"{output}.cif", fmt="cif")

def inspect_zeolite(index: int):
    zeo, lqg = get_zeolite_structure_and_lqg(index)
    output = "{:06d}".format(index)
    zeo.to(f"{output}.cif", fmt="cif")
    draw_bu_contraction(lqg, output)
    bc_embedding(zeo, lqg, output+"_bu")
    return zeo, lqg


In [2]:
zeo42, lqg42 = inspect_zeolite(42)

F:\venv\CrystalGraph\lib\site-packages\pymatgen\analysis\local_env.py:4135: UserWarning: No oxidation states specified on sites! For better results, set the site oxidation states in the structure.
  warnings.warn(
F:\venv\CrystalGraph\lib\site-packages\pymatgen\analysis\local_env.py:3928: UserWarning: CrystalNN: cannot locate an appropriate radius, covalent or atomic radii will be used, this can lead to non-optimal results.
  warnings.warn(


> O96 Si48
> MultiGraph with 144 nodes and 192 edges
> # of cross boundary edges 32
> {'lattice': Lattice
    abc : 9.686 12.226 22.836
 angles : 90.0 90.0 90.0
 volume : 2704.2627780959997
      A : 9.686 0.0 5.930964448270631e-16
      B : 1.966090751494656e-15 12.226 7.486265883187771e-16
      C : 0.0 0.0 22.836
    pbc : True True True}
> Counter({'O Si Si': 96, 'O O O O Si': 48})


In [3]:
zeo43, lqg43 = inspect_zeolite(43)

> O96 Si48
> MultiGraph with 144 nodes and 192 edges
> # of cross boundary edges 28
> {'lattice': Lattice
    abc : 20.634 20.634 8.426
 angles : 90.0 90.0 120.00000000000001
 volume : 3106.8403642483854
      A : 20.634 0.0 1.2634681026803243e-15
      B : -10.316999999999998 17.86956818168811 1.2634681026803243e-15
      C : 0.0 0.0 8.426
    pbc : True True True}
> Counter({'O Si Si': 96, 'O O O O Si': 48})


In [4]:
lqg42 == lqg43

False

In [5]:
lqg42.to_uqg() == lqg43.to_uqg()

False